# Background Information

In this notebook I will set up a basic flow for uploading and dowloading files. Formating datasets and ultimately running cytetype.

In [1]:
# Importing Require Libraries
import cytetype
import scanpy as sc
import session_info
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Session Info
session_info.show()

/Users/alexantill/Göran_Karlsson_Lab/code_benchLLM/.venv/lib/python3.12/site-packages/session_info/main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


In [4]:
# Importing AnnData file
# File Path
file_path_Simone = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Simone_et_al/7d821d98-5b42-4480-8173-641c1b37b237.h5ad")
# AnnData Read In
adata = sc.read_h5ad(file_path_Simone)

In [5]:
# Normalize (10,000 Counts per Cell)
sc.pp.normalize_total(adata, target_sum=1e4)

# Log1p transform
sc.pp.log1p(adata)

/Users/alexantill/Göran_Karlsson_Lab/code_benchLLM/.venv/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:243: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/Users/alexantill/Göran_Karlsson_Lab/code_benchLLM/.venv/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:387: RuntimeWarning: invalid value encountered in log1p
  np.log1p(X, out=X)


In [26]:
# Explore log1p Error (It is due to NaNs)
print("Min:", np.min(adata.X))
print("Any NaNs in sparse data?", np.isnan(adata.X.data).any())
print("Any Infs in sparse data?", np.isinf(adata.X.data).any())

Min: nan
Any NaNs in sparse data? True
Any Infs in sparse data? False


In [6]:
# Find Most Up/Down Regulated Genes 
sc.tl.rank_genes_groups(adata, groupby='cell_type', method='t-test', key_added = 'Rank_Genes_cell_type')  # try 'wilcoxon' if accurate


In [7]:
annotator = cytetype.CyteType(adata, group_key = 'cell_type', rank_key= 'Rank_Genes_cell_type', gene_symbols_column = 'gene_name')

TypeError: CyteType.__init__() got an unexpected keyword argument 'group_key'

In [ ]:
annotator = cytetype.CyteType(adata, cell_group_key = 'cell_type', rank_genes_key= 'Rank_Genes_cell_type', gene_symbols_column_name = 'gene_name')

Calculating expression percentages.


In [14]:
# Run annotation with custom model
adata = annotator.run(
    study_context = "Adult human peripheral blood",
    model_config = [{
        'provider': 'openai',
        'name': 'meta-llama/llama-4-maverick',
        'apiKey': os.environ.get('OPENROUTER_API_KEY'),
        'baseUrl': "https://openrouter.ai/api/v1",
    }],
    auth_token = os.environ.get('NYGEN_API_KEY'), 

)

TypeError: CyteType.run() got an unexpected keyword argument 'study_context'

In [14]:
adata.obs.cell_type

GATCACACACGGGTAA-1_10X_3-rep1     CD4-positive, alpha-beta T cell
GTGTGGCTCTATTGTC-1_10X_3-rep1              CD14-positive monocyte
CTCGAGGGTTCGGTTA-1_10X_3-rep1                       megakaryocyte
TCTACATGTGTCCGGT-1_10X_3-rep1              CD14-positive monocyte
GGTGAAGTCGTTCCTG-1_10X_3-rep1                 natural killer cell
                                               ...               
TTTTGCAAATTT_Scipio-rep2                   CD14-positive monocyte
TTTTGGCCGACG_Scipio-rep2         CD14-low, CD16-positive monocyte
TTTTGGCGGCCC_Scipio-rep2          CD4-positive, alpha-beta T cell
TTTTGTAACTCA_Scipio-rep2                      natural killer cell
TTTTTTTTTTTT_Scipio-rep2                   CD14-positive monocyte
Name: cell_type, Length: 124132, dtype: category
Categories (10, object): ['B cell', 'CD4-positive, alpha-beta T cell', 'CD14-positive monocyte', 'CD14-low, CD16-positive monocyte', ..., 'megakaryocyte', 'natural killer cell', 'plasmacytoid dendritic cell', 'unknown']

In [32]:
adata.obs['CyteType_cell_type']

GATCACACACGGGTAA-1_10X_3-rep1                 T cell
GTGTGGCTCTATTGTC-1_10X_3-rep1               Monocyte
CTCGAGGGTTCGGTTA-1_10X_3-rep1               Platelet
TCTACATGTGTCCGGT-1_10X_3-rep1               Monocyte
GGTGAAGTCGTTCCTG-1_10X_3-rep1    Natural killer cell
                                        ...         
TTTTGCAAATTT_Scipio-rep2                    Monocyte
TTTTGGCCGACG_Scipio-rep2                    Monocyte
TTTTGGCGGCCC_Scipio-rep2                      T cell
TTTTGTAACTCA_Scipio-rep2         Natural killer cell
TTTTTTTTTTTT_Scipio-rep2                    Monocyte
Name: CyteType_cell_type, Length: 124132, dtype: category
Categories (8, object): ['Cytotoxic T cell', 'Dendritic cell', 'Monocyte', 'Naive B cell', 'Natural killer cell', 'Plasmacytoid dendritic cell', 'Platelet', 'T cell']

In [44]:
Author_vs_Llama4Mav = pd.crosstab(adata.obs.cell_type, adata.obs.CyteType_cell_type).T.idxmax()

In [39]:
annotator.cluster_map

{'B cell': '1',
 'CD14-low, CD16-positive monocyte': '2',
 'CD14-positive monocyte': '3',
 'CD4-positive, alpha-beta T cell': '4',
 'cytotoxic T cell': '5',
 'dendritic cell': '6',
 'megakaryocyte': '7',
 'natural killer cell': '8',
 'plasmacytoid dendritic cell': '9',
 'unknown': '10'}

In [48]:
Author_vs_Llama4Mav.reset_index().to_csv("/Users/alexantill/Göran_Karlsson_Lab/benchLLM/Simone_et_al/author_vs_Llama4.csv", index=False)

In [46]:
Author_vs_Llama4Mav

cell_type
B cell                                           B cell
CD4-positive, alpha-beta T cell                  T cell
CD14-positive monocyte                         Monocyte
CD14-low, CD16-positive monocyte               Monocyte
cytotoxic T cell                                 T cell
dendritic cell                                 Monocyte
megakaryocyte                             Megakaryocyte
natural killer cell                 Natural Killer Cell
plasmacytoid dendritic cell                      B cell
unknown                                          B cell
dtype: category
Categories (5, object): ['B cell', 'Megakaryocyte', 'Monocyte', 'Natural Killer Cell', 'T cell']

In [1]:
!uv pip install -U cytetype

Using Python 3.12.10 environment at: /Users/alexantill/Göran_Karlsson_Lab/code_benchLLM/.venv
Resolved 27 packages in 475ms                                        
Prepared 1 package in 27ms                                               
Uninstalled 1 package in 3ms
Installed 1 package in 1ms                                  
 - cytetype==0.2.1
 + cytetype==0.3.1


In [13]:
annotator